In [10]:
import pickle 
import random 
import string 
import numpy as np 
import pandas as pd 
import json
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D, Flatten
from tensorflow.keras.models import Model

import streamlit as st

In [2]:
champ_names = pickle.load(open("data files/champ_names.pkl",'rb'))
responses = pickle.load(open("data files/responses.pkl",'rb'))

data = pd.read_csv('data files/Train_data.csv')

In [3]:
# Tokenizing the inputs

tokenzier = Tokenizer(num_words=2000)
tokenzier.fit_on_texts(data['inputs'])
train = tokenzier.texts_to_sequences(data['inputs'])

# Appling Padding

x_train = pad_sequences(train)

# Encoding

labelEnc = LabelEncoder()
y_train = labelEnc.fit_transform(data['tags'])

# Setting the input shape

input_shape = x_train.shape[1]

In [4]:
# Setting the URLs
build_url = "https://u.gg/lol/champions/{}/build"
patchhis_url = "https://leagueoflegends.fandom.com/wiki/{}/LoL/Patch_history"


def name_find(list1, list2):
    for i in list1: 
        if i not in list1 or i not in list2:
            name = i
    return name

In [5]:
# Defining Vocabulary 

vocabulary = len(tokenzier.word_index)
print("Unique word count:", vocabulary)
output_length = labelEnc.classes_.shape[0]
print("Final output classes:", output_length)

# Defining the model

inp = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(inp)
x = LSTM(20,return_sequences=True)(x)
#x = Dropout(0.5)(x)
x = Flatten()(x)
x = Dense(output_length, activation='softmax')(x)
model = Model(inp,x)

model.load_weights("data files/model_weights.h5")

Unique word count: 86
Final output classes: 10


In [6]:
def predictor():
    
    bot_name = "LOL Bot : "
    print("Hi! I'm the LOL Chatbot")
    
    input_text = input("You : ")
    texts_pred = []
    
    # Preprocessing
    user_input = [wrd.lower() for wrd in input_text if wrd not in string.punctuation]
    user_input = ''.join(user_input)
    texts_pred.append(user_input)

    # Tokenizing and Padding
    user_input = tokenzier.texts_to_sequences(texts_pred)
    user_input = np.array(user_input).reshape(-1)
    user_input = pad_sequences([user_input],input_shape)

    # Prediction
    output = model.predict(user_input, verbose=0)
    output = output.argmax()

    # Assigning the right tag for the prediction
    response_tag = labelEnc.inverse_transform([output])[0]
    
    return response_tag, input_text

In [7]:
def responder(pred):
    
    static_tags = ['greetings', 'tierlist', 'patchnotes', 'bored',  'fun', 'interesting']
    ignore_terms = ["build", "runes", "patches", "patch", "history", "previous", "of", "rune"]
    
    bot_name = "LOL Bot : "
    
    response_tag = pred[0] 
    input_text = pred[1] 
    
    if response_tag == "runes":
        name = name_find(input_text.split(), ignore_terms)
        
        if name.capitalize() in champ_names:            
            print(bot_name, random.choice(responses[response_tag]))
            print(build_url.format(name))
            
        else:
            
            print(bot_name, "OOPS! Give me a valid champ name")
    
    if response_tag == "patchhistory":
        name = name_find(input_text.split(), ignore_terms)
        
        if name.capitalize() in champ_names:            
            print(bot_name, random.choice(responses[response_tag]))
            print(patchhis_url.format(name))
            
        else:
            print(bot_name, "OOPS! Give me a valid champ name")
         
    if response_tag == "broken":
        for i in responses[response_tag]:
            print(i)

    if response_tag in static_tags:
        print(bot_name, random.choice(responses[response_tag]))         


In [11]:
def main():

    # Title of the Page
    st.title('The League of Legends Chat Bot')
    
    #Addding Image for reference
    st.subheader('Upload CSV File with matching columns')
    

In [8]:
while True:

    message = predictor()
    responder(message)
    
    if message[0] == 'goodbye':
        break

Hi! I'm the LOL Chatbot
You : Hey
LOL Bot :  How can I be of help
Hi! I'm the LOL Chatbot
You : lore
LOL Bot :  Heimer created Yi's goggles with 7 lenses for him so he could expand his line of sight
Hi! I'm the LOL Chatbot
You : teemo runes
LOL Bot :  Quick here's the runes page
https://u.gg/lol/champions/teemo/build
Hi! I'm the LOL Chatbot
You : bb
